In [ ]:
PART = 1

In [ ]:
!nvidia-smi
# from google.colab import drive
# drive.mount('/content/drive')

Mon Jul  3 14:08:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    51W / 400W |  13199MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import pandas as pd
df = pd.read_csv(f'/home/daryna-lab-course/text2food/data/tringas_any_class_higher_95_batch_im_download-div-{PART}.csv')
df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,words,matched_text,raw_url,image_name,labels,scores,image_path
0,0,0,0,['meal'],"In Kardamyli three years ago, once I'd discove...",https://thesinglegourmetandtraveller.files.wor...,5729e94a79ae.jpg,"['describing food', 'describing beverages', 'n...","['0.9097504615783691', '0.04694497212767601', ...",/content/drive/MyDrive/text2food/dataset/befor...
1,1,3,3,"['basmati', 'rice']",DO NOT use basmati rice to prepare chitrannam.,https://2.bp.blogspot.com/-spgP6aiCi9U/UBVnSvR...,2465d452ab65.jpg,"['describing food', 'describing beverages', 'n...","['0.8960762619972229', '0.0688355565071106', '...",/content/drive/MyDrive/text2food/dataset/befor...
2,2,7,7,['gumballs'],Gumballs like these are great every day or for...,https://www.candymachines.com/images/bulk_gumb...,b2a8e8cf7a6d.jpg,"['describing food', 'describing beverages', 'n...","['0.8749205470085144', '0.08553430438041687', ...",/content/drive/MyDrive/text2food/dataset/befor...
3,3,8,8,['leaves'],ENGAGING ONE-YEAR-ONLY DESIGN - The 1996 One-O...,https://www.govmint.com/media/catalog/product/...,f79a157be2fa.jpg,"['describing food', 'not describing beverages'...","['0.8120938539505005', '0.08863692730665207', ...",/content/drive/MyDrive/text2food/dataset/befor...
4,4,9,9,"['coriander', 'flavor']","My first couple of bites took me aback, becaus...",https://img.city-cost.com/800x800/40eaed8df91c...,8a3f66a1bba9.jpeg,"['describing food', 'not describing beverages'...","['0.9168951511383057', '0.03275753930211067', ...",/content/drive/MyDrive/text2food/dataset/befor...
...,...,...,...,...,...,...,...,...,...,...
49373,49373,108449,108449,['malt'],A NAS Malt of Jura; good and cheap!,https://static.whiskybase.com/storage/whiskies...,a21ba30f7fa5.jpg,"['describing food', 'describing beverages', 'n...","['0.4994945526123047', '0.4704190790653229', '...",/content/drive/MyDrive/text2food/dataset/befor...
49374,49374,108454,108454,"['chocolate', 'peanut', 'butter', 'recipes']",Instead of letting them derail my progress I'v...,https://mycrazygoodlife.com/wp-content/uploads...,614715119bc7.jpg,"['describing food', 'not describing beverages'...","['0.9306913614273071', '0.036240167915821075',...",/content/drive/MyDrive/text2food/dataset/befor...
49375,49375,108455,108455,['clafoutis'],"If you are new to this site, then you wouldn't...",https://clborah.files.wordpress.com/2016/04/cl...,a926e8aa4e9c.jpg,"['describing food', 'describing beverages', 'n...","['0.7788848280906677', '0.17939694225788116', ...",/content/drive/MyDrive/text2food/dataset/befor...
49376,49376,108456,108456,['rice'],Rice terraces in Northern are slopes claimed f...,https://4.bp.blogspot.com/-qpxJydB3EwM/Wqzk2MF...,b4cde77aaedb.jpg,"['describing food', 'not describing beverages'...","['0.9370181560516357', '0.035220205783843994',...",/content/drive/MyDrive/text2food/dataset/befor...


In [ ]:
import os
!pip install open_clip_torch matplotlib
import numpy as np
import torch
import open_clip
from PIL import Image
import requests
from io import BytesIO
from open_clip import tokenizer
open_clip.list_pretrained()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model, _, preprocess = open_clip.create_model_and_transforms('ViT-L-14', pretrained='datacomp_xl_s13b_b90k',device=device)

model.eval()
context_length = model.context_length
vocab_size = model.vocab_size

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")

Model parameters: 427,616,513


In [ ]:
positives = [
    'A close-up photo of food',
    'A photo of food' , 'A photo of beverage' , 'A photo of breakfast' , 'A photo of lunch' , 'A photo of dinner' , 'A photo of snack' , 'A photo of Appetizer' , 'A photo of meal' ,
      'A photo of nutritional meal' , 'A photo of a drink' ,
             'A photo of traditional food',
             'culinary',
             'A photo of cooking']

negatives = [
     'Not A close-up photo of food',
     'Not A photo of food' , 'Not A photo of beverage' , 'Not A photo of breakfast' , 'Not A photo of lunch' , 'Not A photo of dinner' , 'Not A photo of snack'  , 'Not A photo of Appetizer' , 'Not A photo of meal' ,
     'Not A photo of nutritional meal' , 'Not A photo of a drink' ]

#texts = positives + negatives
texts = positives

In [ ]:
text_tokens = tokenizer.tokenize(texts).to(device)
with torch.no_grad():
      text_features = model.encode_text(text_tokens).float()
text_features /= text_features.norm(dim=-1, keepdim=True)

In [ ]:
#df = df[:10]

In [ ]:
from tqdm import tqdm
image_names= df.image_name.tolist()
image_names = [f"/home/daryna-lab-course/text2food/data/before_clip_1_2_3/before_clip_1_2_3/{i}" for i in image_names][:10]
#images = [Image.open(i) for i in tqdm(image_paths)]

In [ ]:
import torch
from tqdm import tqdm
batch_size = 512
results = []
not_found = []
for i in tqdm(range(0, len(image_names) , batch_size) , desc= 'processing' , total = (len(image_names)// batch_size) +1 ):
  batch= image_names[i: i+batch_size]

  images = []
  for img in batch:
      try:
          images.append(Image.open(img))
      except Exception as e:
          not_found.append(img)  # Add problematic image to the error list


  image_input = torch.tensor(np.stack([preprocess(i) for i in images])).to(device)
  with torch.no_grad():
    image_features = model.encode_image(image_input).float()
  image_features /= image_features.norm(dim=-1, keepdim=True)
  similarity_matrix = torch.mm(image_features, text_features.T)
  for i in images:
    i.close()
  results.extend([{t:score.item() for t,score in zip(texts,i)} for i in similarity_matrix ])


processing: 100%|██████████| 97/97 [2:46:57<00:00, 103.27s/it]


In [ ]:
#batch.index('/content/drive/MyDrive/text2food/dataset/before_clip_1/c9d/c9d9154710f8.jpg')

296

In [ ]:
#batch[296]

'/content/drive/MyDrive/text2food/dataset/before_clip_1/c9d/c9d9154710f8.jpg'

In [ ]:
#type(df['image_path'])

pandas.core.series.Series

In [ ]:
#len(results)

40152

In [ ]:
import json
with open('/home/daryna-lab-course/text2food/data/clip_data.json', 'w') as f:
    json.dump(results, f)

In [ ]:
with open('/home/daryna-lab-course/text2food/data/deleted_images.txt', 'w') as f:
    for line in not_found:
        f.write(f"{line}\n")

In [ ]:
len(not_found)

9226

In [ ]:
df['image_clip_scores'] = results

In [ ]:
import os
os.makedirs('/home/daryna-lab-course/text2food/data/tringas_any_class_higher_95_clip' , exist_ok=True)
df.to_csv(f'/home/daryna-lab-course/text2food/data/tringas_any_class_higher_95_clip/tringas_any_class_higher_95_batch_clip-{PART}.csv')